In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('D:\SoB\MainSoB.csv',sep=",",header=0, index_col=False)
df = df.drop(df.columns[0], axis=1)
Territories_To_Drop = [2, 11, 15, 72, 78]
Commodities_list=[41,11,81,51,88,91,21]
df = df[~df['State.Code'].isin(Territories_To_Drop)]
df = df[df['Commodity.Code'].isin(Commodities_list)]
df['Liability'] = pd.to_numeric(df['Liability'])
df=df[df['Liability']!=0]
#Drops rows with 0 liability... somehow
def add_leading_zero(column):
    max_length = df[column].astype(str).str.len().max()
    return df[column].astype(str).apply( lambda string: string.zfill(max_length) if len(string) < max_length else string)
df['FIPS']=add_leading_zero('State.Code')+add_leading_zero('County.Code')
df['ID']=df['Year'].astype(str)+'_'+df['FIPS']
main=df.copy()

In [12]:
main.isnull().sum()

Year                        0
State.Code                  0
State.Abbr                  0
County.Code                 0
County.Name                 0
Commodity.Code              0
Commodity.Name              0
Insurance.Plan.Code         0
Insurance.Plan.Abbr         0
Coverage.Category           0
Delivery.Type               0
Coverage.Level              0
Policies.Sold.Count         0
Policies.Earning.Premium    0
Policies.Indemnified        0
Units.Earning.Premium       0
Units.Indemnified           0
Quantity.Type               0
Net.Planted.Quantity        0
Net.Endorsed.Acres          0
Liability                   0
Total.Premium               0
Subsidy                     0
State/Provate.Subsidy       0
Additional.Subsidy          0
EFA.Premium.Discount        0
Indemnity.Amount            0
Loss.Ratio                  0
FIPS                        0
ID                          0
dtype: int64

In [13]:
#For simplicity and data coverage, compile policy deductibles together.
# Coverage = 50% => Catastrophic Coverage, else is Buy Up

# Create Dataset of Loss Ratios by Coverage X Commodity X County - Year


temp= main[['ID','Commodity.Code','Coverage.Level','Total.Premium','Indemnity.Amount','Liability']].copy()

# Filter the DataFrame for Coverage.Level = 0.5
coverage_0_5 = temp[temp['Coverage.Level'] == 0.5]
# Group by ID, Commodity.Code, and Coverage.Level = 0.5, and calculate the sums
grouped_0_5 = coverage_0_5.groupby(['ID', 'Commodity.Code', 'Coverage.Level']).sum().reset_index()
# Filter the DataFrame for Coverage.Level != 0.5
coverage_other = temp[temp['Coverage.Level'] != 0.5]
# Group by ID, Commodity.Code, and Coverage.Level != 0.5, and calculate the sums
grouped_other = coverage_other.groupby(['ID', 'Commodity.Code']).sum().reset_index()
# Concatenate the two grouped DataFrames
result = pd.concat([grouped_0_5, grouped_other])

result['Loss.Ratio'] = result['Indemnity.Amount']/result['Total.Premium']
result['Coverage.Level'] = result['Coverage.Level'].astype(str)
result['Coverage.Level'] = np.where(result['Coverage.Level'] == '0.5', 'CAT', 'BUY')
#Calculate Loss Ratio and Label Results

result.head(30)

,ID,Commodity.Code,Coverage.Level,Total.Premium,Indemnity.Amount,Liability,Loss.Ratio
0,1989_01001,41,CAT,14.0,0,281.0,0.000000
1,1989_01003,41,CAT,719.0,0,7939.0,0.000000
2,1989_01005,21,CAT,846.0,8507,11751.0,10.055556
3,1989_01005,41,CAT,70.0,0,791.0,0.000000
4,1989_01031,41,CAT,228.0,0,2576.0,0.000000
5,1989_01033,21,CAT,16336.0,15277,258543.0,0.935174
6,1989_01033,41,CAT,430.0,0,5752.0,0.000000
7,1989_01033,81,CAT,3665.0,0,31002.0,0.000000
8,1989_01039,21,CAT,2413.0,0,30508.0,0.000000
9,1989_01039,41,CAT,164.0,0,1924.0,0.000000


In [22]:
#Creates new dataframe, where each row for an ID has the Loss Ratio for each Coverage X Commodity
wide = result.pivot_table( index=['ID'],columns=['Commodity.Code','Coverage.Level'], values=['Loss.Ratio'])

#wide = wide.reset_index().copy()
wide.head()

Loss.Ratio                                                  \
Commodity.Code         11            21                   41       51       
Coverage.Level        BUY CAT       BUY        CAT       BUY  CAT BUY CAT   
ID                                                                          
1989_01001            NaN NaN  1.122985        NaN  0.000000  0.0 NaN NaN   
1989_01003       5.961480 NaN  0.316592        NaN  0.114442  0.0 NaN NaN   
1989_01005            NaN NaN  1.917953  10.055556  1.622552  0.0 NaN NaN   
1989_01031       3.704942 NaN  4.096840        NaN  0.000000  0.0 NaN NaN   
1989_01033            NaN NaN  1.952855   0.935174  0.000000  0.0 NaN NaN   

                                           
Commodity.Code        81       88  91      
Coverage.Level       BUY  CAT BUY BUY CAT  
ID                                         
1989_01001           NaN  NaN NaN NaN NaN  
1989_01003      1.050901  NaN NaN NaN NaN  
1989_01005           NaN  NaN NaN NaN NaN  
1989_01031      2.603691  NaN NaN NaN NaN  
1989_01033      1.416871  0.0 NaN NaN NaN

In [23]:
# Dataset: By Column Year, Binary =1 if Loss>0
binarywide = wide.applymap(lambda x: 0 if x == 0 else 1 if pd.notnull(x) else np.nan)

binarywide.head()

Loss.Ratio                                                    \
Commodity.Code         11       21        41       51       81       88  91   
Coverage.Level        BUY CAT  BUY  CAT  BUY  CAT BUY CAT  BUY  CAT BUY BUY   
ID                                                                            
1989_01001            NaN NaN  1.0  NaN  0.0  0.0 NaN NaN  NaN  NaN NaN NaN   
1989_01003            1.0 NaN  1.0  NaN  1.0  0.0 NaN NaN  1.0  NaN NaN NaN   
1989_01005            NaN NaN  1.0  1.0  1.0  0.0 NaN NaN  NaN  NaN NaN NaN   
1989_01031            1.0 NaN  1.0  NaN  0.0  0.0 NaN NaN  1.0  NaN NaN NaN   
1989_01033            NaN NaN  1.0  1.0  0.0  0.0 NaN NaN  1.0  0.0 NaN NaN   

                    
Commodity.Code      
Coverage.Level CAT  
ID                  
1989_01001     NaN  
1989_01003     NaN  
1989_01005     NaN  
1989_01031     NaN  
1989_01033     NaN

In [24]:
summary_wide=wide.describe()
print(summary_wide)

                  Loss.Ratio                                            \
Commodity.Code            11                          21                 
Coverage.Level           BUY           CAT           BUY           CAT   
count           52196.000000  42984.000000  15208.000000  13069.000000   
mean                1.129527      0.807223      1.118035      0.701506   
std                 1.643417      2.247141      1.363348      2.115727   
min                 0.000000     -0.249144      0.000000      0.000000   
25%                 0.102387      0.000000      0.136650      0.000000   
50%                 0.541924      0.000000      0.653024      0.013135   
75%                 1.528852      0.581551      1.610391      0.541935   
max                25.204180     59.500000     17.051867     64.871345   

                                                                        \
Commodity.Code            41                          51                 
Coverage.Level           BUY         

In [25]:
summary_binarywide=binarywide.describe()
print(summary_wide)

                  Loss.Ratio                                            \
Commodity.Code            11                          21                 
Coverage.Level           BUY           CAT           BUY           CAT   
count           52196.000000  42984.000000  15208.000000  13069.000000   
mean                1.129527      0.807223      1.118035      0.701506   
std                 1.643417      2.247141      1.363348      2.115727   
min                 0.000000     -0.249144      0.000000      0.000000   
25%                 0.102387      0.000000      0.136650      0.000000   
50%                 0.541924      0.000000      0.653024      0.013135   
75%                 1.528852      0.581551      1.610391      0.541935   
max                25.204180     59.500000     17.051867     64.871345   

                                                                        \
Commodity.Code            41                          51                 
Coverage.Level           BUY         

In [45]:
# Split Up Data by Commodity
# Recover Year / County 
temp = wide.reset_index().copy()
temp['Year']= temp['ID'].str[:4].astype(int)
temp['FIPS'] = temp['ID'].str[5:10]
temp = temp.drop('ID', axis =1)


C:\Users\siegl\AppData\Local\Temp\ipykernel_13408\1663882329.py:6: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  temp = temp.drop('ID', axis =1)


In [83]:
BU11 = temp.iloc[:, [0,13,14]]
CAT11 = temp.iloc[:, [1,13,14]]
BU21= temp.iloc[:, [2,13,14]]
CAT21= temp.iloc[:, [3,13,14]]
BU41= temp.iloc[:, [4,13,14]]
CAT41= temp.iloc[:, [5,13,14]]
BU51= temp.iloc[:, [6,13,14]]
CAT51= temp.iloc[:, [7,13,14]]
BU81= temp.iloc[:, [8,13,14]]
CAT81= temp.iloc[:, [9,13,14]]
BU88= temp.iloc[:, [10,13,14]]
BU91= temp.iloc[:, [11,13,14]]
CAT91= temp.iloc[:, [12,13,14]]
dflist = ['BU11', 'CAT11', 'BU21', 'CAT21', 'BU41', 'CAT41', 'BU51', 'CAT51', 'BU81', 'CAT81', 'BU88', 'BU91', 'CAT91']
for dfname in dflist:
    dataframe= locals()[dfname]
    dataframe.columns = ['LossRatio','Year', 'FIPS']
    dataframe['RunningAvg'] = dataframe.groupby('FIPS', group_keys=True)['LossRatio'].apply(lambda x: x.shift().expanding().mean()).reset_index(level=0, drop=True)


C:\Users\siegl\AppData\Local\Temp\ipykernel_13408\961691729.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['RunningAvg'] = dataframe.groupby('FIPS', group_keys=True)['LossRatio'].apply(lambda x: x.shift().expanding().mean()).reset_index(level=0, drop=True)
C:\Users\siegl\AppData\Local\Temp\ipykernel_13408\961691729.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['RunningAvg'] = dataframe.groupby('FIPS', group_keys=True)['LossRatio'].apply(lambda x: x.shift().expanding().mean()

In [85]:
BU11.tail()

,LossRatio,Year,FIPS,RunningAvg
64588,4.440329,2021,56029,1.595954
64589,0.235386,2021,56031,1.124784
64590,NaN,2021,56037,NaN
64591,NaN,2021,56041,NaN
64592,NaN,2021,56043,0.000000


In [46]:
temp.head()

Loss.Ratio                                                  \
Commodity.Code         11            21                   41       51       
Coverage.Level        BUY CAT       BUY        CAT       BUY  CAT BUY CAT   
0                     NaN NaN  1.122985        NaN  0.000000  0.0 NaN NaN   
1                5.961480 NaN  0.316592        NaN  0.114442  0.0 NaN NaN   
2                     NaN NaN  1.917953  10.055556  1.622552  0.0 NaN NaN   
3                3.704942 NaN  4.096840        NaN  0.000000  0.0 NaN NaN   
4                     NaN NaN  1.952855   0.935174  0.000000  0.0 NaN NaN   

                                           Year   FIPS  
Commodity.Code        81       88  91                   
Coverage.Level       BUY  CAT BUY BUY CAT               
0                    NaN  NaN NaN NaN NaN  1989  01001  
1               1.050901  NaN NaN NaN NaN  1989  01003  
2                    NaN  NaN NaN NaN NaN  1989  01005  
3               2.603691  NaN NaN NaN NaN  1989  01031  
4               1.416871  0.0 NaN NaN NaN  1989  01033

In [60]:

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

C:\Users\siegl\AppData\Local\Temp\ipykernel_13408\888087765.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BU11['RunningAvg'] = BU11.groupby('FIPS', group_keys=True)['LossRatio'].apply(lambda x: x.shift().expanding().mean()).reset_index(level=0, drop=True)


In [80]:
BU11.tail(30)

,LossRatio,Year,FIPS,Avg,RunningAvg
64563,0.993145,2021,55109,1.238889,1.248719
64564,0.605573,2021,55111,0.627943,0.628714
64565,NaN,2021,55113,NaN,NaN
64566,0.229492,2021,55115,1.032579,1.060271
64567,0.024744,2021,55117,0.607235,0.627321
64568,0.000000,2021,55119,0.970629,1.031293
64569,0.000000,2021,55121,0.000000,0.000000
64570,0.000000,2021,55123,1.480484,1.665544
64571,0.329434,2021,55127,0.942708,0.962491
64572,5.554416,2021,55129,1.212152,1.005378


In [92]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits = 5,
              shuffle = True,
              random_state = 582)
    
avg_rmses_LossRatioRunnAvg = []

for dfname in dflist:
    data= locals()[dfname]
    data=data.dropna()
    
    
    
    
    rmse = np.sqrt(mean_squared_error(data['LossRatio'].values, data['RunningAvg'].values))
    avg_rmses.append(rmse)

In [93]:
avg_rmses_LossRatioRunnAvg


[1.8206738946172702,
 2.4564561827483002,
 1.4458522809169585,
 2.2174495152782785,
 1.5993012508086004,
 3.391783200065014,
 1.73712443334229,
 2.271395015102789,
 1.2367770483828282,
 2.033287965632894,
 0.8159991870585502,
 1.8110035168152832,
 2.277591812800116]

In [ ]:
#Baseline Models
#Classification: 

